In [2]:
import pandas as pd

In [11]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [41]:
raw = pd.read_csv('../data/raw/ld_clump_assoc.txt',delimiter='\t')
rename = {'CpG':'cpg',
 'Top SNP':'snp',
 'Beta': 'beta',
 'SE': 'se',
 'Cis/Trans':'cistrans',
 'CpG chr': 'cpg_chr',
 'CpG pos': 'cpg_pos',
 'SNP chr': 'snp_chr',
 'SNP pos': 'snp_pos',
 'P': 'pval'}

raw = data.rename(rename, axis=1)

lst = []
for i in raw['cistrans']:
    if i == 'cis':
        lst.append(True)
    else:
        lst.append(False)
raw['cistrans'] = lst
raw.to_csv('edit_ld_clump_assoc.csv')


In [64]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5']



In [67]:

data = data[data['cistrans']==False]
data = data[data['pval']<minPval]


data['cpg_pos_abs'] = data['cpg_chr'].apply(lambda cpg_chr: chromosome_distance[str(cpg_chr)]) + data['cpg_pos'] # calculate absolute distance of cpg
data['snp_pos_abs'] = data['snp_chr'].apply(lambda snp_chr: chromosome_distance[str(snp_chr)]) + data['snp_pos'] # calculate absolute distance of snp
data['dist'] = abs(data['cpg_pos_abs'] - data['snp_pos_abs']) # calculate distance between pairs
data = data[data['dist']>=minDistance] # filter by min distance

df_g = data.groupby('cpg') # group data by cpgs
df_g = df_g.filter(lambda x: len(x) >= minAssoc) # filter by number of associations per cpg
df_g = df_g.reset_index(drop=True)
    
df_g['id'] = df_g.index 

response = df_g.to_dict('records')  


- Cytoband preprocess

In [3]:
cyto = pd.read_csv('../data/raw/cytobands.csv')

In [7]:
cyto.to_json('cyto.json',orient='records')